In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt

# Load dataset

In [2]:
data = pd.read_csv("artificial_dataset.csv")
data = data.drop("Unnamed: 0", axis=1)
data

,0,1,2,3,4,5,6,7,8,9,label
0,3,3,3,3,3,3,3,3,3,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,3,2,1,0,3,2,1,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,3,3,3,3,3,3,3,3,3,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,3,3,3,3,3,3,3,3,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,1,2,3,3,3,3,3,3,3,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...
99995,1,1,1,1,2,3,0,1,2,3,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
99996,3,2,1,0,3,3,3,3,3,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
99997,0,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
99998,1,3,3,3,3,3,3,3,3,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [3]:
# parse label's str to bit list
y=data['label']  # Labels
y_parse = []
for n,i in enumerate(y):
    y_parse.append([int(j) for j in i.strip("[").strip("]").strip("\n").strip(".").split(", ")])

In [44]:
# drop all zero's data
dr = []
yy = []
for n,row in enumerate(y_parse):
    if all([r==0 for r in row]):
        dr.append(n)
    else:
        yy.append(np.array(row))
        #yy.append(np.sum([bit*2**h for h, bit in enumerate(row)]))
data = data.drop(data.index[dr])

IndexError: index 33052 is out of bounds for axis 0 with size 33052

In [4]:
# shuffling data
data["label"] = y_parse
data = data.sample(frac=1)
X, Y = data.iloc[:,:-1], data["label"]

# Split dataset into training set and test set

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [5]:
# normalization layer
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(X_train))

In [42]:
np.array(y_train).reshape(len(y_train))

ValueError: cannot reshape array of size 23136 into shape (23136,40)

In [6]:
y_train)

70000

### Train linear regressor

In [7]:
linear_model = tf.keras.Sequential([
    normalizer, 
    layers.Dense(units=1)
])

linear_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

In [8]:
%%time
history = linear_model.fit(
    X_train,
    y_train,
    epochs=10,
    # Suppress logging.
    verbose=1,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.2)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [33]:
def plot_loss(history):
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.ylim([0, 10])
    plt.xlabel('Epoch')
    plt.ylabel('Error [MPG]')
    plt.legend()
    plt.grid(True)

In [10]:
plot_loss(history)
plt.ylim(0,0.4)

NameError: name 'history' is not defined

In [ ]:
y_train[0]

In [9]:
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(10, activation='relu'),
      layers.Dense(10, activation='relu'),
      #layers.Dense(10, activation='relu'),
      layers.Dense(40)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

In [10]:
dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 10)               21        
 n)                                                              
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 dense_3 (Dense)             (None, 40)                440       
                                                                 
Total params: 681
Trainable params: 660
Non-trainable params: 21
_________________________________________________________________


In [11]:
%%time
history = dnn_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    verbose=1, epochs=20,workers=8)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [ ]:
plot_loss(history)
plt.ylim(0,0.1)

In [ ]:
pred = dnn_model.predict(X_test)

In [ ]:
from sklearn import metrics

In [ ]:
metrics.r2_score(pred,y_test)

In [ ]:
plt.hist(pred)

In [ ]:
y_test

In [ ]:
(pred > 0.1).any()